In [1]:
import altair as alt
from altair import datum

from src.data.read_dataset import get_processed_dataset
from src.visualization import visualize as viz

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import HTML, Markdown as md

In [2]:
regione = 5

In [3]:
regioni_df = get_processed_dataset('dpc_regioni')
prov_df = get_processed_dataset('dpc_province')

reg_df = regioni_df[regioni_df.codice_regione == regione]
prov_df = prov_df[(prov_df.codice_regione == regione) & (~prov_df.provincia.str.contains('definizione'))]
prov_df['nuovi_casi'] = prov_df.groupby('provincia').totale_casi.diff()
prov_df['incremento'] = 100.0 * prov_df['nuovi_casi'] / prov_df['totale_casi']
today = reg_df.iloc[-1]
yesterday = reg_df.iloc[-2]

In [4]:
md(f"# Riepilogo regione {reg_df.regione.iloc[0]}")

# Riepilogo regione Veneto

In [5]:
reg_df.iloc[-1]

data                          2020-05-11 17:00:00
stato                                         ITA
codice_regione                                  5
regione                                    Veneto
lat                                       45.4349
long                                      12.3385
ricoverati_con_sintomi                        400
terapia_intensiva                              38
totale_ospedalizzati                          438
isolamento_domiciliare                       5022
totale_positivi                              5460
variazione_totale_positivi                   -131
nuovi_positivi                                 19
dimessi_guariti                             11615
deceduti                                     1666
totale_casi                                 18741
tamponi                                    445905
casi_testati                               250175
note_it                                       NaN
note_en                                       NaN


In [6]:
html = viz.tabella_riassuntiva(today, yesterday)
HTML(html)

,Oggi,Variazione
Totale Casi,18741,19
Totale Positivi,5460,-131
Tamponi,445905,6383
Casi Testati,250175.0,2558
Deceduti,1666,9
Dimessi Guariti,11615,141
Terapia Intensiva,38,1
Incremento,0.10%,-0.17
Percentuale Positivi,7.49%,-0.07
Percentuale Nuovi Positivi,0.74%,-1.16


In [7]:
viz.grafici_riassuntivi(reg_df)

alt.VConcatChart(...)

In [8]:
c = alt.Chart(prov_df).mark_line().encode(
        x = 'data:T',
        color = 'provincia:N',
        tooltip = ['data', 'provincia', 'totale_casi', 'nuovi_casi', 'incremento']
    ).properties(
        width=800,
        height=600
    ).interactive()

@interact(logarithm=False)
def plot_province(logarithm):    
    if logarithm:
        return c.encode(y = alt.Y('totale_casi:Q', scale=alt.Scale(type='log'))).transform_filter(datum.totale_casi > 100)
    else:
        return c.encode(y = alt.Y('totale_casi:Q')).transform_filter(datum.totale_casi > 100)

interactive(children=(Checkbox(value=False, description='logarithm'), Output()), _dom_classes=('widget-interac…

In [9]:
c.encode(y = alt.Y(f'incremento:Q', scale=alt.Scale(domain=(0, 10))), row='provincia').properties(
    width=800,
    height=100
)

alt.Chart(...)

In [10]:
latest = prov_df.groupby('provincia').last().reset_index()

alt.Chart(latest).mark_bar().encode(
    x='nuovi_casi:Q',
    y=alt.Y('provincia:N', sort='-x'),
).properties(
    height=300,
    width=800,
    title=f'Nuovi casi per provincia, data = {latest.iloc[0, 1]}'
).interactive()


alt.Chart(...)

In [12]:
alt.Chart(prov_df).mark_line().transform_window(
    avg='mean(nuovi_casi)',
    frame=[0, 3],
    groupby=['provincia']
).encode(
    x = 'data:T',
    facet = alt.Facet('provincia:N', columns=4),
    y = alt.Y('avg:Q', title=''),
    tooltip = ['data', 'provincia', 'totale_casi', 'nuovi_casi', 'incremento']
).properties(
    width=200,
    height=200,
    title={
      "text": ["Nuovi casi positivi per provincia"], 
      "subtitle": ["Dato mediato su 3 giorni"]
    }
).interactive()

alt.Chart(...)